In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', 100)
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [2]:
df.shape

(250, 38)

In [3]:
df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [4]:
# discarding the commas between the actors' full names and getting only the first three names
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])

In [5]:
df['Actors']

0            [Tim Robbins,  Morgan Freeman,  Bob Gunton]
1               [Marlon Brando,  Al Pacino,  James Caan]
2             [Al Pacino,  Robert Duvall,  Diane Keaton]
3        [Christian Bale,  Heath Ledger,  Aaron Eckhart]
4           [Martin Balsam,  John Fiedler,  Lee J. Cobb]
                             ...                        
245           [Ray Milland,  Jane Wyman,  Phillip Terry]
246    [Brie Larson,  John Gallagher Jr.,  Stephanie ...
247      [Cary Grant,  Rosalind Russell,  Ralph Bellamy]
248    [Sissy Spacek,  Jane Galloway Heitz,  Joseph A...
249           [Dev Patel,  Saurabh Shukla,  Anil Kapoor]
Name: Actors, Length: 250, dtype: object

In [6]:
# putting the genres in a list of words
df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

In [7]:
df['Genre']

0                 [crime,  drama]
1                 [crime,  drama]
2                 [crime,  drama]
3        [action,  crime,  drama]
4                 [crime,  drama]
                  ...            
245           [drama,  film-noir]
246                       [drama]
247    [comedy,  drama,  romance]
248           [biography,  drama]
249                       [drama]
Name: Genre, Length: 250, dtype: object

In [8]:
df['Director'] = df['Director'].map(lambda x: x.split(' '))

In [9]:
df['Director']

0                      [Frank, Darabont]
1               [Francis, Ford, Coppola]
2               [Francis, Ford, Coppola]
3                   [Christopher, Nolan]
4                        [Sidney, Lumet]
                     ...                
245                      [Billy, Wilder]
246            [Destin, Daniel, Cretton]
247                      [Howard, Hawks]
248                       [David, Lynch]
249    [Danny, Boyle,, Loveleen, Tandan]
Name: Director, Length: 250, dtype: object

In [10]:
for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [11]:
df['Actors']

0                 [timrobbins, morganfreeman, bobgunton]
1                    [marlonbrando, alpacino, jamescaan]
2                  [alpacino, robertduvall, dianekeaton]
3             [christianbale, heathledger, aaroneckhart]
4                 [martinbalsam, johnfiedler, leej.cobb]
                             ...                        
245                [raymilland, janewyman, phillipterry]
246     [brielarson, johngallagherjr., stephaniebeatriz]
247           [carygrant, rosalindrussell, ralphbellamy]
248    [sissyspacek, janegallowayheitz, josepha.carpe...
249                [devpatel, saurabhshukla, anilkapoor]
Name: Actors, Length: 250, dtype: object

In [12]:
df['Director']

0                  frankdarabont
1             francisfordcoppola
2             francisfordcoppola
3               christophernolan
4                    sidneylumet
                 ...            
245                  billywilder
246          destindanielcretton
247                  howardhawks
248                   davidlynch
249    dannyboyle,loveleentandan
Name: Director, Length: 250, dtype: object

In [13]:
!pip install rake-nltk
from rake_nltk import Rake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
# initializing the new column
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)

In [20]:
key_words_dict_scores

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'mumbai': 3,
             'teen': 3,
             'reflects': 3,
             'upbringing': 1,
             'slums': 1,
             'accused': 1,
             'cheating': 1,
             'indian': 2,
             'version': 2,
             'wants': 1,
             'millionaire': 2,
             '?"': 2})

In [21]:
df.set_index('Title', inplace = True)
df.head()

,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[two, imprisoned, men, bond, number, years, fi..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[aging, patriarch, organized, crime, dynasty, ..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[early, life, career, vito, corleone, 1920s, n..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[menace, known, joker, emerges, mysterious, pa..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[jury, holdout, attempts, prevent, miscarriage..."


In [22]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [23]:
df.head()

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
The Godfather,crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martinbalsam johnfied...


In [24]:
df.shape

(250, 1)

In [ ]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [ ]:
count_matrix

<250x2961 sparse matrix of type '<class 'numpy.int64'>'
	with 5342 stored elements in Compressed Sparse Row format>

In [ ]:
c=count_matrix.todense()

In [ ]:
c

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
type(count_matrix)

scipy.sparse.csr.csr_matrix

In [ ]:
print(count_matrix[0,:])

  (0, 584)	1
  (0, 768)	1
  (0, 1011)	1
  (0, 2678)	1
  (0, 1810)	1
  (0, 306)	1
  (0, 2765)	1
  (0, 1269)	1
  (0, 1733)	1
  (0, 311)	1
  (0, 888)	1
  (0, 2174)	1
  (0, 519)	1
  (0, 655)	1
  (0, 969)	1
  (0, 2481)	1
  (0, 2950)	1
  (0, 59)	1
  (0, 1899)	1


In [ ]:
count.vocabulary_

{'crime': 584,
 'drama': 768,
 'frankdarabont': 1011,
 'timrobbins': 2678,
 'morganfreeman': 1810,
 'bobgunton': 306,
 'two': 2765,
 'imprisoned': 1269,
 'men': 1733,
 'bond': 311,
 'eventual': 888,
 'redemption': 2174,
 'common': 519,
 'decency': 655,
 'finding': 969,
 'solace': 2481,
 'years': 2950,
 'acts': 59,
 'number': 1899,
 'francisfordcoppola': 1007,
 'marlonbrando': 1671,
 'alpacino': 118,
 'jamescaan': 1355,
 'organized': 1932,
 'dynasty': 793,
 'transfers': 2730,
 'control': 553,
 'aging': 82,
 'patriarch': 1972,
 'clandestine': 475,
 'empire': 846,
 'reluctant': 2193,
 'son': 2492,
 'robertduvall': 2264,
 'dianekeaton': 710,
 'vito': 2854,
 'corleone': 563,
 'tightens': 2670,
 '1920s': 8,
 'new': 1870,
 'york': 2953,
 'portrayed': 2057,
 'career': 391,
 'family': 934,
 'syndicate': 2608,
 'grip': 1131,
 'expands': 908,
 'michael': 1750,
 'early': 795,
 'life': 1575,
 'action': 56,
 'christophernolan': 464,
 'christianbale': 458,
 'heathledger': 1179,
 'aaroneckhart': 31,
 

In [ ]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.15789474, 0.13764944, ..., 0.05263158, 0.05263158,
        0.05564149],
       [0.15789474, 1.        , 0.36706517, ..., 0.05263158, 0.05263158,
        0.05564149],
       [0.13764944, 0.36706517, 1.        , ..., 0.04588315, 0.04588315,
        0.04850713],
       ...,
       [0.05263158, 0.05263158, 0.04588315, ..., 1.        , 0.05263158,
        0.05564149],
       [0.05263158, 0.05263158, 0.04588315, ..., 0.05263158, 1.        ,
        0.05564149],
       [0.05564149, 0.05564149, 0.04850713, ..., 0.05564149, 0.05564149,
        1.        ]])

In [ ]:
pd.DataFrame(cosine_sim)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249
0,1.000000,0.157895,0.137649,0.083771,0.111283,0.046829,0.095673,0.244558,0.044992,0.046829,0.051299,0.000000,0.000000,0.045883,0.052632,0.170406,0.00000,0.039936,0.210526,0.051299,0.097823,0.091766,0.089984,0.050063,0.046829,0.055641,0.108148,0.052632,0.054074,0.143509,0.000000,0.000000,0.054074,0.000000,0.108148,0.045883,0.000000,0.040555,0.051299,0.046829,0.105263,0.046829,0.000000,0.050063,0.050063,0.000000,0.000000,0.054074,0.052632,0.046829,...,0.108148,0.041204,0.000000,0.000000,0.000000,0.111283,0.051299,0.000000,0.091766,0.086711,0.052632,0.048912,0.146735,0.052632,0.102598,0.047836,0.055641,0.047836,0.118470,0.000000,0.052632,0.043355,0.047836,0.054074,0.045883,0.000000,0.220755,0.048912,0.054074,0.044992,0.000000,0.061314,0.059235,0.047836,0.057354,0.057354,0.052632,0.050063,0.000000,0.059235,0.000000,0.000000,0.000000,0.057354,0.050063,0.057354,0.048912,0.052632,0.052632,0.055641
1,0.157895,1.000000,0.367065,0.125656,0.166924,0.046829,0.047836,0.146735,0.044992,0.046829,0.051299,0.041885,0.000000,0.045883,0.052632,0.213007,0.00000,0.039936,0.157895,0.051299,0.146735,0.137649,0.134976,0.050063,0.046829,0.055641,0.162221,0.052632,0.054074,0.143509,0.000000,0.000000,0.054074,0.000000,0.162221,0.045883,0.047836,0.000000,0.102598,0.046829,0.052632,0.140488,0.000000,0.100125,0.050063,0.000000,0.000000,0.054074,0.052632,0.093659,...,0.162221,0.041204,0.000000,0.000000,0.000000,0.055641,0.051299,0.000000,0.045883,0.130066,0.052632,0.048912,0.146735,0.105263,0.153897,0.095673,0.055641,0.000000,0.177705,0.000000,0.052632,0.043355,0.047836,0.000000,0.045883,0.000000,0.220755,0.097823,0.054074,0.044992,0.000000,0.061314,0.059235,0.047836,0.057354,0.057354,0.052632,0.050063,0.000000,0.059235,0.000000,0.051299,0.000000,0.114708,0.050063,0.057354,0.048912,0.052632,0.052632,0.055641
2,0.137649,0.367065,1.000000,0.109545,0.145521,0.040825,0.041703,0.127920,0.078446,0.040825,0.044721,0.000000,0.000000,0.080000,0.045883,0.259973,0.00000,0.000000,0.137649,0.134164,0.127920,0.120000,0.156893,0.043644,0.040825,0.048507,0.141421,0.091766,0.047140,0.125109,0.000000,0.000000,0.047140,0.000000,0.141421,0.040000,0.041703,0.000000,0.134164,0.040825,0.045883,0.122474,0.000000,0.043644,0.043644,0.036515,0.000000,0.047140,0.045883,0.122474,...,0.141421,0.071842,0.000000,0.043644,0.000000,0.048507,0.044721,0.000000,0.040000,0.113389,0.045883,0.042640,0.127920,0.091766,0.178885,0.083406,0.048507,0.000000,0.154919,0.000000,0.045883,0.075593,0.041703,0.000000,0.040000,0.000000,0.230940,0.127920,0.047140,0.039223,0.041703,0.053452,0.103280,0.125109,0.100000,0.050000,0.045883,0.043644,0.044721,0.051640,0.000000,0.000000,0.000000,0.150000,0.087287,0.100000,0.042640,0.045883,0.045883,0.048507
3,0.083771,0.125656,0.109545,1.000000,0.088561,0.037268,0.076139,0.077850,0.071611,0.111803,0.040825,0.066667,0.074536,0.036515,0.041885,0.101710,0.08165,0.063564,0.083771,0.040825,0.116775,0.073030,0.071611,0.079682,0.037268,0.088561,0.086066,0.041885,0.043033,0.152277,0.000000,0.045644,0.043033,0.043033,0.086066,0.036515,0.114208,0.032275,0.081650,0.037268,0.167542,0.037268,0.043033,0.039841,0.039841,0.066667,0.000000,0.086066,0.041885,0.074536,...,0.086066,0.098374,0.037268,0.119523,0.038925,0.044281,0.040825,0.039841,0.036515,0.069007,0.041885,0.038925,0.077850,0.041885,0.081650,0.114208,0.044281,0.000000,0.094281,0.114208,0.041885,0.034503,0.076139,0.043033,0.073030,0.000000,0.105409,0.038925,0.086066,0.035806,0.000000,0.048795,0.047140,0.038069,0.045644,0.045644,0.041885,0.039841,0.000000,0.047140,0.038925,0.081650,0.000000,0.045644,0.039841,0.045644,0.038925,0.041885,0.041885,0.044281
4,0.111283,0.166924,0.145521,0.0885

In [ ]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    print(top_10_indexes)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [ ]:
recommendations('Inception')

[25, 55, 199, 203, 159, 165, 40, 16, 89, 74]


['Interstellar',
 'Aliens',
 'Guardians of the Galaxy',
 'Guardians of the Galaxy Vol. 2',
 'Mad Max: Fury Road',
 'Spider-Man: Homecoming',
 'The Prestige',
 'The Matrix',
 'Batman Begins',
 '2001: A Space Odyssey']

In [ ]:
indices[indices == 'Inception'].index[0]

12

In [ ]:
pd.Series(cosine_sim[indices[indices == 'Fargo'].index[0]])

0      0.132453
1      0.220755
2      0.192450
3      0.105409
4      0.140028
5      0.039284
6      0.040129
7      0.123091
8      0.037743
9      0.039284
10     0.043033
11     0.000000
12     0.000000
13     0.038490
14     0.044151
15     0.178685
16     0.000000
17     0.000000
18     0.132453
19     0.043033
20     0.164122
21     0.153960
22     0.150970
23     0.041996
24     0.039284
25     0.046676
26     0.136083
27     0.044151
28     0.045361
29     0.120386
         ...   
220    0.044151
221    0.109109
222    0.040129
223    0.000000
224    0.038490
225    0.000000
226    0.222222
227    0.082061
228    0.045361
229    0.037743
230    0.000000
231    0.051434
232    0.099381
233    0.040129
234    0.096225
235    0.048113
236    0.044151
237    0.083992
238    0.000000
239    0.099381
240    0.000000
241    0.000000
242    0.000000
243    0.048113
244    0.041996
245    0.048113
246    0.041030
247    0.044151
248    0.044151
249    0.046676
Length: 250, dtype: floa